In [1]:
# creating results after creating new fxn that utilizes the ratio of the number 
# of components, instead of the number of proteins in a cluster

In [2]:
# importing libraries

# standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# library used for functional enrichment analysis
from func_e.FUNC_E import FUNC_E  # a method that can do funtional enrichment analysis!
import func_e.vocabs.all as vocabs

In [4]:
# following code is necessary as some functions used in the funcE class produce future warnings but i can't fix it as it is not my code
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
# importing the custom classes i built
from matrix_class import ProteinMatrix # ppi matrix 
from cluster_class import AllClusters # dictionary to hold all clusters (in form number of cluster : list of proteins in that cluster)
from degreelist_class import DegreeList # creates a list of all proteins in order of their degree

In [6]:
# helper functions for setting up program
from recipe_utils import initialize_matrix_clusters_degreelist
from recipe_utils import find_clusters_and_proteins_together

# helper functions for functional enrichment
from recipe_utils import print_querylist_of_clusters_to_file
from recipe_utils import print_protein_background_to_file
from recipe_utils import create_term_mapping_list
from recipe_utils import get_initialized_fe

In [7]:
matrix, clusters, degreelist = initialize_matrix_clusters_degreelist("data/interactions/4_sparsified.txt", "data/clusters/4_sparsified_clusters_s100.txt")

In [8]:
qualifying_clusters, qualifying_proteins = find_clusters_and_proteins_together(matrix, clusters, degreelist, cluster_ratio=0, cluster_constant=2,use_sqrt=True, protein_ratio=1, protein_constant=0)

print(qualifying_proteins)

{}


In [9]:

qualifying_clusters5, qualifying_proteins5 = find_clusters_and_proteins_together(matrix, clusters, degreelist, cluster_ratio=0, cluster_constant=2,use_sqrt=True, protein_ratio=.5, protein_constant=0)
# cluster const = 2 -> must have at least 2 components

print(qualifying_proteins5)

{}


In [10]:

qualifying_clusters3, qualifying_proteins3 = find_clusters_and_proteins_together(matrix, clusters, degreelist, cluster_ratio=0, cluster_constant=2,use_sqrt=True, protein_ratio=.25, protein_constant=0)

print(qualifying_proteins3)

{15: [12578, 12577, 12573, 12551, 12524, 12521, 12519, 12508, 12499, 12489, 12479, 12477, 12474, 12468, 12466, 12455, 12454, 12453, 12451, 12440, 12436, 12435, 12404, 12397, 12387, 12386, 12377, 12373, 12372, 12370, 12363, 12340, 12329, 12326, 12320, 12311, 12309, 12306, 12304, 12302, 12289, 12284, 12261, 12260, 12259, 12258, 12253, 12252, 12225, 12211, 12202, 12201, 12195, 12183, 12159, 12156, 12153, 12130, 12118, 12112, 12111, 12046, 12043, 12036, 12022, 12019, 12008, 12004, 11993, 11988, 11976, 11959, 11956, 11955, 11951, 11937, 11932, 11929, 11924, 11919, 11896, 11895, 11879, 11856, 11854, 11852, 11841, 11829, 11828, 11807, 11802, 11783, 11780, 11776, 11756, 11745, 11725, 11724, 11705, 11704, 11703, 11693, 11678, 11659, 11657, 11654, 11642, 11629, 11626, 11610, 11608, 11603, 11596, 11538, 11529, 11522, 11521, 11518, 11498, 11467, 11416, 11411, 11399, 11381, 11368, 11366, 11358, 11356, 11349, 11342, 11334, 11332, 11303, 11297, 11286, 11284, 11283, 11280, 11267, 11264, 11248, 11240, 

In [18]:
qualifying_clusters3, qualifying_proteins3 = find_clusters_and_proteins_together(matrix, clusters, degreelist, cluster_ratio=0, cluster_constant=2,use_sqrt=True, protein_ratio=0, protein_constant=2)



In [17]:
print(qualifying_proteins3)

{}


In [11]:
# use functions to print info to files: (annoying, but need to get info into format used for funcE package)

# 1) a background protein list to be used for functional enrichment analysis
background_proteins_filepath = "data/node_lists/4_sparse_nodelist.txt"
print_protein_background_to_file(matrix, background_proteins_filepath) 


In [12]:
# TODO: don't have this yet

# 2) file containing protein to go term
term_mapping_filepath = "demo_data/term_mapping.txt"
create_term_mapping_list("demo_data/dream3_go_results.tsv", term_mapping_filepath)

# 3) not a file, but need to establish a background of all GO terms! (to be used for functional enrichment analysis)
background_go_terms = vocabs.getTerms(['GO'])

In [13]:
# continue printing info to files to be used in functional enrichment analysis:

# 4) print original clusters to a file
og_query_filepath = 'demo_data/original_clusters.txt'
print_querylist_of_clusters_to_file(clusters, clusters.get_all_cluster_labels(),og_query_filepath)

# 5) print updated clusters to a file (just append the function to include the dictionary of qualifying proteins)
updated_query_filepath = 'demo_data/updated_clusters.txt'
print_querylist_of_clusters_to_file(clusters, clusters.get_all_cluster_labels(), updated_query_filepath, qualifying_proteins)

In [14]:
original_fe = get_initialized_fe(background_proteins_filepath, term_mapping_filepath, termlist = background_go_terms)

original_fe.importFiles({'query': og_query_filepath })
original_fe.run(cluster=False)

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
# do functional enrichment on updated clusters!!

updated_fe_all_proteins = get_initialized_fe(background_proteins_filepath, term_mapping_filepath, termlist = background_go_terms)

updated_fe_all_proteins.importFiles({'query': updated_query_filepath })
updated_fe_all_proteins.run(cluster=False)

In [ ]:
# print data by the numbers!
print(f"number of clusters that were updated with proteins: {len(qualifying_proteins.keys())}\n")

# clusters that are functionally enriched
print(f"number of functionally enriched clusters (modules) out of {len(qualifying_proteins.keys())} original clusters: {original_fe.enrichment['Module'].nunique()}")
print(f"number of functionally enriched clusters after addition of all proteins: {updated_fe_all_proteins.enrichment['Module'].nunique()}")

print(f"")


# significant P-values!!!
print(f"number of significant pvals in original clusters before updating: {original_fe.enrichment['Fishers_pvalue'].count()}")
print(f"number of significant pvals in clusters with all proteins added: {updated_fe_all_proteins.enrichment['Fishers_pvalue'].count()}")